In [ ]:
import sys
import serial
import signal
#import random
from threading import Thread

In [ ]:
def get_input(channel):
    return received[channel].averaged_value()

def signal_handler(signal, frame):
    ser.close()
    print( 'closed port. exit.' )
    sys.exit(0)

def init_serial():
    ser = serial.Serial(port = PORT, baudrate = BAUD_RATE)
    signal.signal(signal.SIGINT, signal_handler)
    return ser

PORT = '/dev/ttyACM3'
BAUD_RATE = 9600
NUM_CHANNELS = 6

WIDTH, HEIGHT = 800, 600
SIZE = (WIDTH, HEIGHT)
BLACK = (0,0,0)
RED = (255,0,0)

ser = init_serial()
#serial_thread = Thread(target=receive_data)
#serial_thread.daemon = True
#serial_thread.start()

In [ ]:
def receiving(ser):
    global last_received
    buffer = ''

    while True:
        buffer += ser.read(ser.inWaiting()).decode("utf-8")
        if '\n' in buffer:
            last_received, buffer = buffer.split('\n')[-2:]

Thread(target=receiving, args=(ser,)).start()

In [ ]:
import time
data = []

for i in range(1000):
    data += last_received
    time.sleep(0.01)

In [ ]:
with open("lucas-2.log", "w") as f:
    f.write(data)

In [ ]:
def serial_input(timeout=1000):
    global last_received
    try:
        #ser.reset_input_buffer()
        rec = last_received
        index = 0
        data = [int(i) for i in rec.split(';')[:-1]]
        #print(data)

        #mins = data[::3]
        #maxs = data[2::3]
        #vals = data[1::3]
        mins = []
        maxs = []
        vals = data
        return mins, maxs, vals
    except:
        return [], [], []

_,_,vals = serial_input()

print(vals)

In [ ]:
data = []
for i in range(1000):
    _,_,vals = serial_input()
    data.append(vals)
    time.sleep(0.1)

In [ ]:
import numpy as np
import pandas as pd

np.array(data)
pd.DataFrame(data).to_csv("lucas.csv")


In [ ]:
from IPython import display
import seaborn as sns
from pylab import *
import time

%matplotlib inline

def get_data():
    vals = []
    while not (len(vals) == 6):
        _,_,vals = serial_input()
    
    return range(6), np.array(vals) / 1023.

x = []
y = []
#offset = None

#mins = np.zeros((6,)) + 1000
#maxs = np.zeros((6,)) + 0.01

for i in range(1000):
    x, y = get_data()
    if i == 0:
        print("set offset")
        offset = y.copy()
    y -= offset
    mins = np.minimum(y, mins)
    maxs = np.maximum(y, maxs)
    y = (y - mins) / (maxs - mins)
    
    if i % 10:
        plt.gca().cla() 
        plt.plot(x,y[::-1],label='test',marker="x")
        plt.legend()
        plt.xticks(x)
        plt.yticks([0,1])
        display.clear_output(wait=True)
        display.display(plt.gcf())
        #display.display(y)
        #display.display(offset)
        #display.display(mins)
        #display.display(maxs)
        time.sleep(0.5)
        

In [ ]:
ser.close()